# Pipeline
This notebook takes LCO data straight from the [Archive](https://archive.lco.global), placed into the `data` directory, and performs photometry on the supernova within the images. Optionally, this can perform difference imaging to remove the background galaxy.

To run the notebook step-by step, click on the code cell below, and press `Shift-Enter` on your keyboard to run the cells in sequence. Output files will arrive in the `output` directory after you've finished - you can download the PDF files to see how the photometry went.

If you're on Google Colab or haven't yet completed installation, you'll need to run the cell below 

In [ ]:
!pip install --upgrade --quiet pip setuptools wheel
!pip install --quiet -r ../requirements.txt

Next, we prepare the pipeline by removing any old outputs, moving your data files to a working directory, and uncompressing any compressed files so that the pipeline can read them.

In [2]:
import glob
from astropy.io import fits
import logging
import os
import shutil

logging.getLogger().setLevel("INFO")


def cleanup():
    psf_outputs = ["*_seq.txt", "template_*.fits"]
    if os.path.exists("PSF_output/"):
        shutil.rmtree("PSF_output/")

    for output in psf_outputs:
        if outlist := glob.glob(output):
            for item in outlist:
                os.remove(item)

    if fitslist := glob.glob("*[.fits,.fz]"):
        for file in fitslist:
            os.remove(file)


# Clean up any old runs
cleanup()

# Copy frames from data to here - temporarily.
alldata = glob.glob("../data/*[.fits,.fz]")

local_paths = []

for file in alldata:
    potential_path = os.path.join(os.getcwd(), os.path.basename(file))
    local_paths.append(potential_path)

    if not os.path.exists(potential_path):
        shutil.copy(file, os.getcwd())

# Unpack the FITS files - these are compressed by default to save bandwidth while transmitting.
for file in alldata:
    if ".fz" in file:
        new_file = os.path.basename(file).replace(".fits.fz", ".fits")
        hdul = fits.open(file)
        hdul.verify('fix')
        datahdu = hdul[1].data
        headerhdu = hdul[1].header
        hdu = fits.PrimaryHDU(datahdu, header=headerhdu)
        hdu.writeto(new_file, overwrite=True)

Filename: ../data/tfn0m436-sq33-20240212-0177-e91.fits.fz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SCI           1 CompImageHDU    268   (2400, 2400)   float32   
  2  CAT           1 BinTableHDU    127   136R x 40C   [D, D, D, D, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, D, D, D, D]   
  3  BPM           1 CompImageHDU      9   (2400, 2400)   uint8   
  4  ERR           1 CompImageHDU     10   (2400, 2400)   float32   
Filename: ../data/tfn0m436-sq33-20240215-0126-e91.fits.fz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  SCI           1 CompImageHDU    268   (2400, 2400)   float32   
  2  CAT           1 BinTableHDU    127   101R x 40C   [D, D, D, D, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, D, D, D, D]   
  3  BPM           1 CompI

### Configuring the pipeline
We need to set a few things before running the pipeline:
- `TARGET_RA`, `TARGET_DEC`: the right ascension and declination of the supernova, so we know where it should be in the image. If you're running this in Colab, you can use the boxes that appear to the right to set these

In [3]:
# Here we need to set some parameters, so the code knows where to look
TARGET_RA = 155.457239894  # @param {type:"number"}
TARGET_DEC = 56.9279057494  # @param {type:"number"}

# To enable image subtraction, add `--sub` to the list of options below
# To enable cosmic ray cleaning, add `--clean` to the list of options below

!python \
    ../photometry-sans-frustration/psf.py \
    -c $TARGET_RA $TARGET_DEC \
    --clean  \
    --quiet \
    --savefigs;

[1000]
#################################################
#                                               #
#  Welcome to PSF: Photometry Sans Frustration  #
#                    (V1.7)                     #
#      Written by Matt Nicholl (2015-2022)      #
#                                               #
#################################################
Figure(1400x700)

File: tfn0m436-sq33-20240221-0126-e91.fits
Filter found in header: rp
Calibrating to filter: r

File: tfn0m419-sq32-20240225-0116-e91.fits
Filter found in header: rp
Calibrating to filter: r

File: tfn0m436-sq33-20240212-0175-e91.fits
Filter found in header: gp
Calibrating to filter: g

File: tfn0m419-sq32-20240225-0115-e91.fits
Filter found in header: gp
Calibrating to filter: g

File: tfn0m436-sq33-20240210-0089-e91.fits
Filter found in header: ip
Calibrating to filter: i

File: elp0m414-sq31-20240309-0139-e91.fits
Filter found in header: rp
Calibrating to filter: r

File: tfn0m419-sq32-20240206-0185-e91.fits
Filter

In [4]:
logging.info("Cleaning up and placing things in correct place")

for path in local_paths:
    os.remove(path)

# move the PSF output to the output folder
for file in glob.glob("PSF_output/*"):
    shutil.move(file, "../outputs/")

cleanup()

INFO:root:Cleaning up and placing things in correct place
